# Iterative Construction of a Penalised Vine Structure
This notebook iteratively estimate the quantile.

#### Libraries

In [1]:
import openturns as ot
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

random_state = 123
np.random.seed(random_state)

#### Model function
This example consider the simple additive example.

In [2]:
from dependence.tests.test_functions import func_overflow, margins_overflow, var_names_overflow

def sum_func(x, a=None):
    """Additive example
    """        
    n, dim = x.shape
    if a is None:
        a = np.ones((dim, 1))

    if a.ndim == 1:
        a = a.reshape(-1, 1)
        assert a.shape[0] == dim, "Shape not good"
    elif a.ndim > 2:
        raise AttributeError('Dimension problem for constant a')
        
    return np.dot(x, a)

test_func = func_overflow

#### Dimension and margins
We first define the problem dimension and the margins

In [3]:
if test_func == sum_func:
    dim = 6
    margins = [ot.Normal()]*dim
else:
    margins = margins_overflow
    dim = len(margins)

We chose the coefficients of the variables throught the additive function.

In [4]:
if test_func == sum_func:
    coeficients = np.logspace(0., 3., dim+1, endpoint=False)[1:]

    n_plot = 10000
    x = np.asarray(ot.ComposedDistribution(margins).getSample(n_plot))
    y = sum_func(x, coeficients)

    fig, axes = plt.subplots(dim, 1, sharex=True, sharey=True, figsize=(4, 2*dim))
    for i in range(dim):
        ax = axes[i]
        ax.plot(x[:, i], y, '.')
        ax.set_xlabel(r'$X_%d$' % (i+1), fontsize=12)
        ax.set_ylabel(r'$y$', fontsize=12)
    fig.tight_layout()

#### Copula families
We now consider only Gaussian dependencies for this example

In [5]:
families = np.zeros((dim, dim), dtype=int)
for i in range(1, dim):
    for j in range(i):
        families[i, j] = 1

## Estimations
We create an instance of the main class for conservative estimate, and we define a q_func object for the quantile as a quantity of interest

In [6]:
from dependence import ConservativeEstimate, quantile_func

alpha = 0.99
def q_func(x, axis=1):
    return - quantile_func(alpha)(x, axis=axis)

quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)

First, we compute the quantile at independence

In [7]:
n = 5000
indep_result = quant_estimate.independence(n_input_sample=n, q_func=q_func, random_state=random_state)

In [8]:
indep_result.compute_bootstrap(1000)
boot_std = indep_result.bootstrap_sample.std()
boot_mean = indep_result.bootstrap_sample.mean()
print('Quantile at independence: %.2f with a C.O.V at %.1f %%' % (boot_mean, abs(boot_std/boot_mean)*100.))

Quantile at independence: 1.69 with a C.O.V at 16.5 %


### Iterative Approach
Now lets see how good we can be with the iterative appraoch.

In [ ]:
from dependence.iterative_vines import iterative_vine_minimize

p_max = 10
K = None
verbose = True
grid_type = 'lhs'
n_add_pairs = 2
n_remove_pairs = 0

quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)
worst_quantities, selected_pairs, removed_pairs = iterative_vine_minimize(quant_estimate, n, K, 
                                                                          p_max, grid_type=grid_type, 
                                                                          q_func=q_func, 
                                                                          n_add_pairs=n_add_pairs,
                                                                          n_remove_pairs=n_remove_pairs,
                                                                          verbose=True)

Worst quantile of [(1, 0)] at -4.09585316998
Worst quantile of [(2, 0)] at 0.422900142915
Worst quantile of [(2, 1)] at 0.42599868132
Worst quantile of [(3, 0)] at 1.04581771498
Worst quantile of [(3, 1)] at 1.20803592031
Worst quantile of [(3, 2)] at 1.98027241895
Worst quantile of [(4, 0)] at 1.77151417369
Worst quantile of [(4, 1)] at 1.33625014603
Worst quantile of [(4, 2)] at 1.77664656003
Worst quantile of [(4, 3)] at 1.85369648151
Worst quantile of [(5, 0)] at 2.12415750499
Worst quantile of [(5, 1)] at 1.87145218573
Worst quantile of [(5, 2)] at 1.63505170732
Worst quantile of [(5, 3)] at 2.14414505265
Worst quantile of [(5, 4)] at 1.8552458763
Worst quantile of [(6, 0)] at 1.71479305405
Worst quantile of [(6, 1)] at 1.52551089727
Worst quantile of [(6, 2)] at 1.35607056731
Worst quantile of [(6, 3)] at 1.80699258289
Worst quantile of [(6, 4)] at 1.61360237238
Worst quantile of [(6, 5)] at 1.87437278309
Worst quantile of [(7, 0)] at 1.29160122848
Worst quantile of [(7, 1)] at 1

### Grid Search Approach
Firstly, we consider a grid search approach in order to compare the perfomance with the iterative algorithm.

In [ ]:
K = 2000
grid_type = 'lhs'
quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)
grid_result = quant_estimate.gridsearch_minimize(n_dep_param=K, n_input_sample=n, grid_type=grid_type, 
                                                 q_func=q_func, random_state=random_state)

In [ ]:
families

In [ ]:
min_result = grid_result.min_result
min_result.compute_bootstrap(1000)
boot_std = min_result.bootstrap_sample.std()
boot_mean = min_result.bootstrap_sample.mean()
print('Worst Quantile: %.2f with a C.O.V at %.1f %%' % (boot_mean, abs(boot_std/boot_mean)*100.))
print('Cost = %d' % (K*n))

In [ ]:
if dim == 2:
    fig, ax = plt.subplots()
    ax.plot(grid_result.dep_params, grid_result.quantities)